# Section 1: Data Import and Mini Feature Exploration

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math

In [3]:
digits = pd.read_csv("digits.csv", header=None)
digits = digits.sample(frac=1, random_state=200).reset_index(drop=True)

In [4]:
digits.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
784,0,1,10,13,2,0,0,0,0,10,...,0,0,0,9,13,11,10,9,0,2
417,0,0,6,9,11,9,0,0,0,13,...,0,0,0,3,13,12,7,1,0,3
1748,0,0,11,12,0,0,0,0,0,7,...,0,0,0,12,14,13,12,5,0,2
424,0,0,0,1,7,15,11,0,0,0,...,0,0,0,0,0,9,6,0,0,9
1162,0,0,4,14,16,14,1,0,0,2,...,0,0,0,2,9,15,16,8,0,1


In [5]:
# dimensions of dataset
digits.shape

(1797, 65)

In [6]:
print("Sample size:", digits.shape[0])
print("Number of features in dataset:", digits.shape[1])

Sample size: 1797
Number of features in dataset: 65


In [7]:
# global variables
sample_size = digits.shape[0]
num_of_features = digits.shape[1] - 1
k = 10
train_test_split_ratio = 0.2

In [8]:
X = digits.iloc[:,:-1] # features
y = digits.iloc[:,-1]  # labels

In [9]:
X.shape

(1797, 64)

In [10]:
X.describe()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
count,1797.0,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,...,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000,1797.000000
mean,0.0,0.303840,5.204786,11.835838,11.848080,5.781859,1.362270,0.129661,0.005565,1.993879,...,3.725097,0.206455,0.000556,0.279354,5.557596,12.089037,11.809126,6.764051,2.067891,0.364496
std,0.0,0.907192,4.754826,4.248842,4.287388,5.666418,3.325775,1.037383,0.094222,3.196160,...,4.919406,0.984401,0.023590,0.934302,5.103019,4.374694,4.933947,5.900623,4.090548,1.860122
min,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,0.000000,1.000000,10.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,11.000000,10.000000,0.000000,0.000000,0.000000
50%,0.0,0.000000,4.000000,13.000000,13.000000,4.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,4.000000,13.000000,14.000000,6.000000,0.000000,0.000000
75%,0.0,0.000000,9.000000,15.000000,15.000000,11.000000,0.000000,0.000000,0.000000,3.000000,...,7.000000,0.000000,0.000000,0.000000,10.000000,16.000000,16.000000,12.000000,2.000000,0.000000
max,0.0,8.000000,16.000000,16.000000,16.000000,16.000000,16.000000,15.000000,2.000000,16.000000,...,16.000000,13.000000,1.000000,9.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000


---

# Section 2: Building our Gaussian Naive Bayes from Scratch

In [11]:
def separateByClass(dataset):
    separated = dict()
    for i in range(dataset.shape[0]):
        vector = dataset.iloc[i,:]
        class_value = vector.iloc[-1]
        if (class_value not in separated):
            separated[class_value] = list()
        separated[class_value].append(vector)
    return separated

In [12]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))

In [13]:
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([(x-avg)**2 for x in numbers]) / float(len(numbers)-1)
    return math.sqrt(variance)

In [14]:
def summarizeDataset(dataset):
    summaries = [(mean(column), stdev(column), len(column)) for column in zip(*dataset)]
    del(summaries[-1])
    return summaries

In [15]:
def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = dict()
    for class_value, rows in separated.items():
        summaries[class_value] = summarizeDataset(rows)
    return summaries

In [16]:
def calculateProbability(x, mean, stdev):
    eps = 1e-4
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2) + eps)))
    return (1/(math.sqrt(2*math.pi)*stdev + eps))*exponent

def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev, count = classSummaries[i]
            x = inputVector.iloc[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

In [17]:
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

In [18]:
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(testSet.shape[0]):
        result = predict(summaries, testSet.iloc[i,:])
        predictions.append(result)
    return predictions

In [19]:
def getError(testSet, predictions):
    wrong = 0
    for x in range(testSet.shape[0]):
        if testSet.iloc[x, -1] != predictions[x]:
            wrong += 1
    return (wrong/float(len(testSet)))

In [20]:
summaries = summarizeByClass(digits)
predictions = getPredictions(summaries, digits)
error = getError(digits, predictions)
print('Error when trained and tested on total dataset:', error)

Error when trained and tested on total dataset: 0.16917084028937118


---

# Section 3: First we do 80-20 train test split

In [21]:
train_test_split_ratio = 0.8

In [22]:
def split_train_test(data, ratio):
    m = data.shape[0]
    cut = int(m * ratio)
    
    train = data.iloc[0:cut,:]
    test = data.iloc[cut:,:]
    
    return train, test

In [23]:
train, test = split_train_test(digits, train_test_split_ratio)

In [24]:
summaries = summarizeByClass(train)
predictions = getPredictions(summaries, train)
error = getError(train, predictions)
print("Train Error: %.4f" % error)

predictions = getPredictions(summaries, test)
error = getError(test, predictions)
print("Test Error: %.4f" % error)

Train Error: 0.1726
Test Error: 0.1861


---

# Section 4: Now we apply same steps for 10-fold cross validation

In [25]:
# global variables
k_fold = 10
seed = 200

In [26]:
# randomise the dataset (with seed for reproducing)
digits = digits.sample(frac=1, random_state=seed).reset_index(drop=True)

# break down indices in 10 folds and save it
folds_indices = []
start_index = 0
steps = sample_size / k_fold
for i in range(k_fold):
    end_index = start_index + steps
    folds_indices.append([round(start_index), round(end_index)])
    start_index = end_index

folds_indices

[[0, 180],
 [180, 359],
 [359, 539],
 [539, 719],
 [719, 898],
 [898, 1078],
 [1078, 1258],
 [1258, 1438],
 [1438, 1617],
 [1617, 1797]]

In [27]:
kfold_train_errors = []
kfold_test_errors = []

for i in range(len(folds_indices)):    
    test = digits.iloc[folds_indices[i][0]:folds_indices[i][1],:]
    train = digits.drop(test.index)
    
    predictions = getPredictions(summaries, train)
    train_error = getError(train, predictions)
    kfold_train_errors.append(train_error)

    predictions = getPredictions(summaries, test)
    test_error = getError(test, predictions)
    kfold_test_errors.append(test_error)

In [28]:
kfold_train_errors

[0.17501546072974644,
 0.17552533992583436,
 0.17068645640074212,
 0.17501546072974644,
 0.17428924598269468,
 0.17996289424860853,
 0.18119975262832405,
 0.17316017316017315,
 0.17367119901112485,
 0.17439703153988867]

In [29]:
kfold_test_errors

[0.17777777777777778,
 0.17318435754189945,
 0.21666666666666667,
 0.17777777777777778,
 0.18435754189944134,
 0.13333333333333333,
 0.12222222222222222,
 0.19444444444444445,
 0.18994413407821228,
 0.18333333333333332]

In [30]:
np.mean(kfold_train_errors)

0.17529230143568833

In [31]:
np.mean(kfold_test_errors)

0.17530415890751086

---

# Section 5: We explore with Scikit-learn and compare with our code

In [32]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [33]:
# Train-test with scikit LogReg
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=train_test_split_ratio, random_state=0)

classifier = GaussianNB()
classifier.fit(X_train, y_train)
print("Train:", (1-classifier.score(X_train, y_train)))
print("Test:", (1-classifier.score(X_test, y_test)))

Train: 0.09749303621169914
Test: 0.15368567454798332


In [34]:
# Kfold with scikit LogReg
from sklearn.model_selection import KFold

kf = KFold(n_splits=k)
kf.get_n_splits(X)
fold_index = 1

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)    
    print("Fold:", fold_index, ", Train error:", (1-classifier.score(X_train, y_train)), ", Test error:", (1-classifier.score(X_test, y_test)))
    fold_index = fold_index + 1

Fold: 1 , Train error: 0.13914656771799627 , Test error: 0.1166666666666667
Fold: 2 , Train error: 0.15708101422387133 , Test error: 0.1611111111111111
Fold: 3 , Train error: 0.1558441558441559 , Test error: 0.16666666666666663
Fold: 4 , Train error: 0.14409400123685834 , Test error: 0.19444444444444442
Fold: 5 , Train error: 0.1323438466295609 , Test error: 0.19444444444444442
Fold: 6 , Train error: 0.1341991341991342 , Test error: 0.15000000000000002
Fold: 7 , Train error: 0.1385281385281385 , Test error: 0.16666666666666663
Fold: 8 , Train error: 0.13164400494437578 , Test error: 0.15083798882681565
Fold: 9 , Train error: 0.13720642768850433 , Test error: 0.17877094972067042
Fold: 10 , Train error: 0.1440049443757726 , Test error: 0.13407821229050276
